Learn to add two 2-digit numbers with fixed-size padded at the right blocks.

In [1]:
from gptbench import Sample, LogFlag, Train, empty_config

To create train and validation dataset:
python prepare_addition.py ../data/add2.txt 2 --sep="\n" --split=0.9

Creates add2.train.txt and add2.val.txt

In [2]:
with open('../data/add2.val.txt', 'r', newline=None) as f:
    val_data = f.read()
val_data[:500]

'90+0=90\n90+1=91\n90+2=92\n90+3=93\n90+4=94\n90+5=95\n90+6=96\n90+7=97\n90+8=98\n90+9=99\n90+10=100\n90+11=101\n90+12=102\n90+13=103\n90+14=104\n90+15=105\n90+16=106\n90+17=107\n90+18=108\n90+19=109\n90+20=110\n90+21=111\n90+22=112\n90+23=113\n90+24=114\n90+25=115\n90+26=116\n90+27=117\n90+28=118\n90+29=119\n90+30=120\n90+31=121\n90+32=122\n90+33=123\n90+34=124\n90+35=125\n90+36=126\n90+37=127\n90+38=128\n90+39=129\n90+40=130\n90+41=131\n90+42=132\n90+43=133\n90+44=134\n90+45=135\n90+46=136\n90+47=137\n90+48=138\n90+49=139\n90+50=140\n90+51=141\n'

In [3]:
ben = Train('add2pad', log_mask=LogFlag.ALL)

# set datasets
ben.set_datasets('padlinechar', train_path='../data/add2.train.txt', val_path='../data/add2.val.txt')

# set config settings
cfg = empty_config()
cfg.train.log_period=0
cfg.model.set(n_layer=6, n_head=6, n_embd=90, block_size=16)
cfg.sample.set(top=1, max_batch_size=256) # top_k(1) - always pick the best item
cfg.train.set(sample_period=-5)
cfg.trainer.set(batch_size=128)

# and init a new model with config
if ben.can_resume():
    ben.init_resume(cfg)
else:
    ben.init_new(cfg)
# print(do.get_config().dump(1))

New random seed 768391787
Loading checkpoint from ./models/add2pad/
Checkpoint: iter=1900 (27.022 epoch), loss train=0.4691 val=0.7730 eval->0.7730
Dataset train_path: ../data/add2.train.txt, val_path: ../data/add2.val.txt, train_split: None, vocab_size: 13
Model params: 0.59M


In [4]:
#ben.line_dataset_split_qa(ben.val_dataset, 0, 20, sep='=')

In [15]:
ds = ben.val_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)

def test(q,a,g):
    res = float(a == g)
    if not res:
        print(f"{q}: {a} != {g}")
    return res
    
ben.measure_accuracy(q,a, test_fn=None)

0.542

In [7]:
ds = ben.train_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)
ben.measure_accuracy(q,a)

0.9272222222222222

In [14]:
q=['30+16=', '91+2=', '11+11=']
a=['46','93','22']
ben.measure_accuracy(q,a)

0.6666666666666666

In [4]:
start_text='30+16=|92+17=|11+11='
ans=[]
ben.sample(start_text, dest=ans, emit_start=False)
ans

['46', '109', '22']

In [7]:
start_text='1+1='
ans=[]
ben.sample(start_text, dest=ans, emit_after='=')
ans

['2']

In [17]:
ben.state['n_samples']/128

1900.0

In [ ]:
ben.train(iter_count=10000)

Training
Resuming optimizer state
Batches per epoch: 70
.CUDA max memory used: 170.00M
...................................................................................................iter 2000 (28.444 epoch): loss train=0.4626, val=0.8079, eval->0.8079
21+4=25
....................................................................................................iter 2100 (29.867 epoch): loss train=0.4588, val=0.8446, eval->0.8446
....................................................................................................iter 2200 (31.289 epoch): loss train=0.4554, val=0.9151, eval->0.9151
....................................................................................................iter 2300 (32.711 epoch): loss train=0.4531, val=0.8183, eval->0.8183
....................................................................................................iter 2400 (34.133 epoch): loss train=0.4496, val=0.8165, eval->0.8165
........................................................

In [64]:
do.sample(start_text="10+20=")

10+20=30


In [56]:
do.sample(start_text="94+97=", emit_start=False)

191


In [5]:
def test_accuracy(first_number, second_number):
    qa = []
    for a in range(100):
        for b in range(100):
            qa.append((f'{a}+{b}=', f'{a+b}'))

    qa=qa[first_number*100:second_number*100]
    
    sep = do.get_config().sample.start_text_sep
    start_text = sep.join([q for q,a in qa])

    ans = []
    do.sample(start_text, dest=ans, emit_after='=')

    corr=0
    for i in range(len(qa)):
        q,a = qa[i]
        #print(q,a,ans[i])
        if a == ans[i]:
            corr+=1
        
    print(f'{corr/len(qa):.3f}')


In [7]:
test_accuracy(90, 100)

0.422


In [8]:
test_accuracy(0, 10)

0.537


In [9]:
test_accuracy(10, 90)

0.621
